# BERTのサンプルコード
ディープラーニングにおける自然言語処理の王様であるBERTに関するコードを作成したので，載せてみました．大まかな流れは以下のようになっております．

1. テキストデータ(ラベル付き)の読み込み
2. 前処理(文書ごとにモデルに適した形に変換)
3. データセット作成
4. 2値分類用のモデルを読み込み
5. 学習実行
6. 精度評価


## ライブラリ作成

In [1]:
import random
import pandas as pd
import numpy as np

import torch
import transformers
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer,BertForSequenceClassification, AdamW, BertConfig

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold

## データ取得

Twitterのテキストを対象に，ポジティブかネガティブについて2値分類を行った．
ポジティブ⇒0，ネガティブ⇒1

今回は下記のテキストデータを対象としてみました！！

In [2]:
df_mazu_kanji = pd.read_json('不味.json')[["text", "評価"]]
df_mazu_kanji

,text,評価
0,そこそこ美味しいけどな?そんな不味そうに見える?,1
1,昨日コンビニでCHABAAのスイカジュース買ったん。飲んだん。くっそ不味い!!!!いや、これ...,1
2,不味っ(&gt;_&lt;),0
3,一回本場に行って不味さを体験してみたいんだがな...,0
4,子供の頃は赤だし不味すぎぃ!って感じで初めて白味噌の味噌汁飲んだらうますぎて感動した思い出。...,1
...,...,...
995,突発性の難聴になってしまい大量の薬を処方されたんだけど薬が不味すぎることによるストレスで完全...,0
996,【食レポ5教科編】英語・・・発音アクセントが不味いが、その他は美味しい。数学・・・全体的に美...,1
997,今日は雨やから買い物行くの面倒でUber使って、自分の好きな福ちゃんラーメンの炒飯頼んだんだ...,1
998,Twitter画像表示とかダメになってる鯖関東近郊にある会社のは色々不味そう,0


## Bertに適した処理を実施

In [ ]:
# データの抽出
sentences = df_mazu_kanji["text"].values
labels = df_mazu_kanji["評価"].values

In [ ]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME, mecab_kwargs={"mecab_option": mecab_path})

# 元の文章
print('original:', sentences[0])

# Tokenizer：単語単位で分割された文章
print('tokenized:', tokenizer.tokenize(sentences[0]))

# Token-id：単語をIDに置き換えた文章
print('Token IDs:', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

In [ ]:
# 最大単語数の確認
max_len = []

# 1文づつ処理
for sent in sentences:
    
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))

# 最大の値を確認
print('最大単語数：', max(max_len))

print('上記の最大単語数にspecial token ([CLS], [SEP])の+2をした値が最大単語数')

## 前処理

In [ ]:
input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,       # Special Tokenの追加
                        max_length = 107,                # 文章の長さを固定(Padding/Trancatinating)
                        pad_to_max_length = True,        # Paddingで埋める
                        return_attention_mask = True,    # Attention mask作成
                        return_tensors = 'pt'            # Pytorch tensorsで返す
                    )
    
    # 単語IDを取得
    input_ids.append(encoded_dict['input_ids'])
    
    # Attention mask取得
    attention_masks.append(encoded_dict['attention_mask'])
    
# リストに入ったtensorを縦方向(dim=0)へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tensor型に変換
labels = torch.tensor(labels)

# 確認
print('original:', sentences[0])
print('Token IDs:', input_ids[0])

## データセットの作成

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 80%地点のIDを取得
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数：{}'.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

## モデルの読み込み

In [ ]:
# BertForSequenceClassification：学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,                     # モデル指定
    num_labels = 2,                 # ラベル数：2
    output_attentions = False,     # アテンションベクトルを出力するか
    output_hidden_states = False,  # 隠れ層を出力するか
)

# モデルをGPUへ転送
#model.cuda()

## 訓練フェーズ

In [226]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]
        optimizer.zero_grad()
        loss = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels).loss
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
        
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in validation_dataloader:
            b_input_ids = batch[0]
            b_input_mask = batch[1]
            b_labels = batch[2]
            with torch.no_grad():
                loss = model(b_input_ids, 
                                      token_type_ids = None,
                                      attention_mask=b_input_mask,
                                      labels=b_labels).loss
            val_loss += loss.item()
    return val_loss

## 学習の実行

In [ ]:
# 学習の実行
max_epoch = 4 #エポック回数
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = validation(model)
    train_loss_.append(train_)
    test_loss_.append(test_)

## 検証フェーズ

In [228]:
model.eval()

logits_df = pd.DataFrame(np.zeros((0, 2)), columns=['logit_0', 'logit_1'])
pred_df = pd.DataFrame(np.zeros((0, 1)), columns=['pred_label'])
label_df = pd.DataFrame(np.zeros((0, 1)), columns=['true_label'])
accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

for batch in validation_dataloader:
    b_input_ids = batch[0]
    b_input_mask = batch[1]
    b_labels = batch[2]
    
    with torch.no_grad():
        
        #学習済みモデルによる予測結果をpredsで取得
        preds = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask)

    
    temp_logits_df = pd.DataFrame(preds[0].numpy(), columns=['logit_0', 'logit_1'])

    temp_pred_df = pd.DataFrame(np.argmax(preds[0].numpy(), axis=1), columns=['pred_label'])

    temp_label_df = pd.DataFrame(b_labels.numpy(), columns=['true_label'])
    
    temp_accuracy_df = pd.concat([temp_logits_df, temp_pred_df, temp_label_df], axis=1)

    accuracy_df = pd.concat([accuracy_df, temp_accuracy_df]).reset_index(drop=True)
    
accuracy_df["pred_label"] = accuracy_df["pred_label"].astype(int)
accuracy_df["true_label"] = accuracy_df["true_label"].astype(int)

In [229]:
accuracy_df

,logit_0,logit_1,pred_label,true_label
0,1.464973,-1.348462,0,0
1,-3.179623,2.706445,1,1
2,-3.252740,2.803764,1,1
3,-3.022099,2.686529,1,1
4,-3.105004,2.739986,1,1
...,...,...,...,...
195,-3.236476,2.715956,1,1
196,-3.152592,2.840008,1,1
197,-3.183203,2.814145,1,1
198,-3.227024,2.699448,1,1


## 精度確認

In [230]:
cm = confusion_matrix(accuracy_df["true_label"], accuracy_df["pred_label"], labels=[1, 0])
print("混同行列")
print(cm)


#精度評価
print('正解率：{}'.format(accuracy_score(accuracy_df["true_label"], accuracy_df["pred_label"])))
print('適合率：{}'.format(precision_score(accuracy_df["true_label"], accuracy_df["pred_label"])))
print('再現率：{}'.format(recall_score(accuracy_df["true_label"], accuracy_df["pred_label"])))
print('F1：{}'.format(f1_score(accuracy_df["true_label"], accuracy_df["pred_label"])))

混同行列
[[114   4]
 [ 26  56]]
正解率：0.85
適合率：0.8142857142857143
再現率：0.9661016949152542
F1：0.8837209302325583


## モデル保存

学習済みのモデルを保存する

In [194]:
model_path = 'model.pth'
torch.save(model.to('cpu').state_dict(), model_path)